# Transformer 구현하기
현재 가장 좋은 성능을 내는 모델이 transformer 기반의 Pretrained 모델. [Attention is All You Need](https://arxiv.org/abs/1706.03762) 논문 참조 할수 있다.


## 1. Vocab
SentencePiece를 활용해 만든 vocab을 이용해 텍스트를 tensor로 변경

In [ ]:
# vocab loading
vocab_file = "<path of data>/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

# 입력 texts
lines = [
  "겨울은 추워요.",
  "감기 조심하세요."
]

# text를 tensor로 변환
inputs = []
for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  inputs.append(torch.tensor(ids))
  print(pieces)

# 입력 길이가 다르므로 입력 최대 길이에 맟춰 padding(0)을 추가 해 줌
inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
# shape
print(inputs.size())
# 값
print(inputs)


실행결과
```
['▁겨울', '은', '▁추', '워', '요', '.']
['▁감', '기', '▁조', '심', '하', '세', '요', '.']
torch.Size([2, 8])
tensor([[3091, 3604,  206, 3958, 3760, 3590,    0,    0],
        [ 212, 3605,   53, 3832, 3596, 3682, 3760, 3590]])
```

## 2. Embedding
Transformer의 Embedding은 **Input Embedding**과 **Position Embedding** 두가지를 합해서 사용.  
### 2.1 Input Embedding  
Embedding은 입력 토큰을 vector로 형태로 변환
    1) innputs에 대한 embedding 값 input_embs를 구한다.

In [ ]:
n_vocab = len(vocab) # vocab count
d_hidn = 128 # hidden size
nn_emb = nn.Embedding(n_vocab, d_hidn) # embedding 객체

input_embs = nn_emb(inputs) # input embedding
print(input_embs.size())

위와 같이 input(2,8)에 대한 Embedding 값 input_embs(2,8,128) shape을 갖는다

In [ ]:
torch.Size([2, 8, 128])


### 2.2 Position Embedding  
position **encoding** 값을 구하기 위한 과정
    - 각 position 별로 angle 값을 구한다
    - 구해진 angle 중 짝수 index의 값에 대한 sin 값을 구한다.
    - 구해진 angle 중 홀수 index의 값에 대한 cos 값을 구한다.

In [ ]:
""" sinusoid position embedding """
def get_sinusoid_encoding_table(n_seq, d_hidn):
    def cal_angle(position, i_hidn):
        return position / np.power(10000, 2 * (i_hidn // 2) / d_hidn)
    def get_posi_angle_vec(position):
        return [cal_angle(position, i_hidn) for i_hidn in range(d_hidn)]

    sinusoid_table = np.array([get_posi_angle_vec(i_seq) for i_seq in range(n_seq)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # even index sin 
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # odd index cos

    return sinusoid_table

**Position Encoding 구하는 방법**

In [ ]:
n_seq = 64
pos_encoding = get_sinusoid_encoding_table(n_seq, d_hidn)

print (pos_encoding.shape) # 크기 출력
plt.pcolormesh(pos_encoding, cmap='RdBu')
plt.xlabel('Depth')
plt.xlim((0, d_hidn))
plt.ylabel('Position')
plt.colorbar()
plt.show()


**Position Encoding 값으로 Position Embedding 생성**  
    - 구해진 Position encoding 값을 이용해 position embedding을 생성. 학습되는 값이 아니므로 freeze 옵션은 True로 설정
    - 입력 inputs과 동일한 크기를 갖는 position 값을 구한다.
    - input 값 중 pad(0)값을 찾는다.
    - position 값중 pad 부분은 0으로 변경
    - position 값에 해당하는 embedding 값을 구한다 

In [ ]:
pos_encoding = torch.FloatTensor(pos_encoding)
nn_pos = nn.Embedding.from_pretrained(pos_encoding, freeze=True)

positions = torch.arange(inputs.size(1), device=inputs.device, dtype=inputs.dtype).expand(inputs.size(0), inputs.size(1)).contiguous() + 1
pos_mask = inputs.eq(0)

positions.masked_fill_(pos_mask, 0)
pos_embs = nn_pos(positions) # position embedding

print(inputs)
print(positions)
print(pos_embs.size())

위에서 구한 **input_embs**와 **pos_embs**를 더하면 transformer에 입력할 input이 된다.

In [ ]:
input_sums = input_embs + pos_embsa

## 3. Scaled Dot Product Attention
![](https://paul-hyun.github.io/assets/2019-12-19/scale_dot_product_attention.png)

### 3.1 입력값
입력값은 Q(query), K(key), V(Value) 그리고 attention mask로 구성되어 있다. 입력 값중 K,V는 같아야하고, Q,K,V가 모두 동일한 경우 self attention이라 한다.

In [ ]:
Q = input_sums
K = input_sums
V = input_sums
attn_mask = inputs.eq(0).unsqueeze(1).expand(Q.size(0), Q.size(1), K.size(1))
print(attn_mask.size())
print(attn_mask[0])

### 3.2 MatMul Q, K-transpose


In [ ]:
scores = torch.matmul(Q, K.transpose(-1, -2))
print(scores.size())
print(scores[0])

### 3.3 Scale

In [ ]:
d_head = 64
scores = scores.mul_(1/d_head**0.5)
print(scores.size())
print(scores[0])

### 3.3 Mask(opt)

In [ ]:
scores.masked_fill_(attn_mask, -1e9)
print(scores.size())
print(scores[0])

### 3.4 Softmax

In [ ]:
attn_prob = nn.Softmax(dim=-1)(scores)
print(attn_prob.size())
print(attn_prob[0])

### 3.5 Matmul(attn_prob, V)
위 수식 중 5번 attention_probablity * V

In [ ]:
context = torch.matmul(attn_prob, V)
print(context.size())

### 3.6 클래스로 통합

In [ ]:
""" scale dot product attention """
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_head):
        super().__init__()
        self.scale = 1 / (d_head ** 0.5)
    
    def forward(self, Q, K, V, attn_mask):
        # (bs, n_head, n_q_seq, n_k_seq)
        scores = torch.matmul(Q, K.transpose(-1, -2)).mul_(self.scale)
        scores.masked_fill_(attn_mask, -1e9)
        # (bs, n_head, n_q_seq, n_k_seq)
        attn_prob = nn.Softmax(dim=-1)(scores)
        # (bs, n_head, n_q_seq, d_v)
        context = torch.matmul(attn_prob, V)
        # (bs, n_head, n_q_seq, d_v), (bs, n_head, n_q_seq, n_v_seq)
        return context, attn_prob

## 4 Multi Head Attention
![](https://paul-hyun.github.io/assets/2019-12-19/multi_head_attention.png)

### 입력값
Q, K, V, attn_mask는 ScaledDotProductAttention과 동일하고, head의 갯수는 2개, head의 dimension은 64이다.

In [ ]:
Q = input_sums
K = input_sums
V = input_sums
attn_mask = inputs.eq(0).unsqueeze(1).expand(Q.size(0), Q.size(1), K.size(1))

batch_size = Q.size(0)
n_head = 2

### Multi Head Q, K, V
위 그림 수식 중 1번 Q를 여러개의 head로 나누는 과정

In [ ]:
W_Q = nn.Linear(d_hidn, n_head * d_head)
W_K = nn.Linear(d_hidn, n_head * d_head)
W_V = nn.Linear(d_hidn, n_head * d_head)

# (bs, n_seq, n_head * d_head)
q_s = W_Q(Q)
print(q_s.size())
# (bs, n_seq, n_head, d_head)
q_s = q_s.view(batch_size, -1, n_head, d_head)
print(q_s.size())
# (bs, n_head, n_seq, d_head)
q_s = q_s.transpose(1,2)
print(q_s.size())

Q값이 head 단위로 나누어 졌다
```
torch.Size([2, 8, 128])
torch.Size([2, 8, 2, 64])
torch.Size([2, 2, 8, 64])
```

위 과정을 한줄로 표현하면

In [2]:
# (bs, n_head, n_seq, d_head)
q_s = W_Q(Q).view(batch_size, -1, n_head, d_head).transpose(1,2)
# (bs, n_head, n_seq, d_head)
k_s = W_K(K).view(batch_size, -1, n_head, d_head).transpose(1,2)
# (bs, n_head, n_seq, d_head)
v_s = W_V(V).view(batch_size, -1, n_head, d_head).transpose(1,2)
print(q_s.size(), k_s.size(), v_s.size())

NameError: name 'W_Q' is not defined

Attention Mask도 Multi Head로 변경
```
torch.Size([2, 8, 8])
torch.Size([2, 2, 8, 8])
```

### Attention
위 그림 수식 중 4번 Attenstion 과정으로, 앞 부분의 Scaled Dot Product Attention을 사용.

In [ ]:
scaled_dot_attn = ScaledDotProductAttention(d_head)
context, attn_prob = scaled_dot_attn(q_s, k_s, v_s, attn_mask)
print(context.size())
print(attn_prob.size())

위 과정에서 multi head에 대한 attention을 구했다.
```
torch.Size([2, 2, 8, 64])
torch.Size([2, 2, 8, 8])
```
  
### Concat
위 그림 수식 중 5번 Concat 과정

In [ ]:
# (bs, n_seq, n_head * d_head)
context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_head * d_head)
print(context.size())

concat을 통해 multi head를 한개로 합쳤다.
```
torch.Size([2, 8, 128])
```

### Linear
위 그림 수식 중 6번 Linear 과정

In [ ]:
linear = nn.Linear(n_head * d_head, d_hidn)
# (bs, n_seq, d_hidn)
output = linear(context)
print(output.size())

입력 Q와 동일한 Shape을 가진 Multi Head Attention이 구해졌다.
```
torch.Size([2, 8, 128])
```

### Class
위 절차를 하나의 클래스 형태로 구성

In [ ]:
""" multi head attention """
class MultiHeadAttention(nn.Module):
    def __init__(self, d_hidn, n_head, d_head):
        super().__init__()
        self.d_hidn = d_hidn
        self.n_head = n_head
        self.d_head = d_head

        self.W_Q = nn.Linear(d_hidn, n_head * d_head)
        self.W_K = nn.Linear(d_hidn, n_head * d_head)
        self.W_V = nn.Linear(d_hidn, n_head * d_head)
        self.scaled_dot_attn = ScaledDotProductAttention(d_head)
        self.linear = nn.Linear(n_head * d_head, d_hidn)
    
    def forward(self, Q, K, V, attn_mask):
        batch_size = Q.size(0)
        # (bs, n_head, n_q_seq, d_head)
        q_s = self.W_Q(Q).view(batch_size, -1, self.n_head, self.d_head).transpose(1,2)
        # (bs, n_head, n_k_seq, d_head)
        k_s = self.W_K(K).view(batch_size, -1, self.n_head, self.d_head).transpose(1,2)
        # (bs, n_head, n_v_seq, d_head)
        v_s = self.W_V(V).view(batch_size, -1, self.n_head, self.d_head).transpose(1,2)

        # (bs, n_head, n_q_seq, n_k_seq)
        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.n_head, 1, 1)

        # (bs, n_head, n_q_seq, d_head), (bs, n_head, n_q_seq, n_k_seq)
        context, attn_prob = self.scaled_dot_attn(q_s, k_s, v_s, attn_mask)
        # (bs, n_head, n_q_seq, h_head * d_head)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_head * self.d_head)
        # (bs, n_head, n_q_seq, e_embd)
        output = self.linear(context)
        # (bs, n_q_seq, d_hidn), (bs, n_head, n_q_seq, n_k_seq)
        return output, attn_prob

## 5. Masked Multi-Head Attention
Masked Multi-Head Attention은 Multi Head Attention과 attention mask를 제외한 부분은 모두 동일하다.

### 입력값

In [ ]:
""" attention decoder mask """
def get_attn_decoder_mask(seq):
    subsequent_mask = torch.ones_like(seq).unsqueeze(-1).expand(seq.size(0), seq.size(1), seq.size(1))
    subsequent_mask = subsequent_mask.triu(diagonal=1) # upper triangular part of a matrix(2-D)
    return subsequent_mask


Q = input_sums
K = input_sums
V = input_sums

attn_pad_mask = inputs.eq(0).unsqueeze(1).expand(Q.size(0), Q.size(1), K.size(1))
print(attn_pad_mask[1])
attn_dec_mask = get_attn_decoder_mask(inputs)
print(attn_dec_mask[1])
attn_mask = torch.gt((attn_pad_mask + attn_dec_mask), 0)
print(attn_mask[1])

batch_size = Q.size(0)
n_head = 2

pad mask, decoder mask 그리고 이 둘을 합한 attention mask를 확인 할 수 있다.
```
tensor([[False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False,  True,  True]])
tensor([[0, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[False,  True,  True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True],
        [False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False,  True,  True]])
```

### Multi-Head Attention
Multi-head attention과 동일하므로, 위에서 선언한 MultiHeadAttention클래스를 바로 호출

In [13]:
attention = MultiHeadAttention(d_hidn, n_head, d_head)
output, attn_prob = attention(Q, K, V, attn_mask)
print(output.size(), attn_prob.size())

NameError: name 'MultiHeadAttention' is not defined

## 6. FeedForwar
![](https://paul-hyun.github.io/assets/2019-12-19/feed-forward.png)

### f1(Linear)
위 그림 수식 중 1번 f1(Linear) 과정

In [ ]:
conv1 = nn.Conv1d(in_channels=d_hidn, out_channels=d_hidn * 4, kernel_size=1)
# (bs, d_hidn * 4, n_seq)
ff_1 = conv1(output.transpose(1, 2))
print(ff_1.size())

입력에 비해 hidden dimension이 4배 커졌다. out_channels = d_hidn*4 으로 출력이 커졌다.
```
torch.Size([2, 512, 8])
```

### Activation(relu or gelu)
위 그림 수식 중 2번 Activation(relu or gelu)과정. 논문이 발표될 당시 relu를 사용했지만 이후 gelu가 성능에 더 좋다는 것이 발견
![](https://paul-hyun.github.io/assets/2019-12-19/activation.png)

In [ ]:
# active = F.relu
active = F.gelu
ff_2 = active(ff_1)

### f3(Linear)
위 그림 수식 중 3번 f3(Linear) 과정.

In [ ]:
conv2 = nn.Conv1d(in_channels=d_hidn * 4, out_channels=d_hidn, kernel_size=1)
ff_3 = conv2(ff_2).transpose(1, 2)
print(ff_3.size())

입력과 동일한 shape으로 변경된 결과를 확인
```
torch.Size([2, 8, 128])
```

### Class 
위 절차를 하나의 클래스 형태로 구성

In [ ]:
""" feed forward """
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_hidn):
        super().__init__()

        self.conv1 = nn.Conv1d(in_channels=self.config.d_hidn, out_channels=self.config.d_hidn * 4, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=self.config.d_hidn * 4, out_channels=self.config.d_hidn, kernel_size=1)
        self.active = F.gelu

    def forward(self, inputs):
        # (bs, d_ff, n_seq)
        output = self.active(self.conv1(inputs.transpose(1, 2)))
        # (bs, n_seq, d_hidn)
        output = self.conv2(output).transpose(1, 2)
        # (bs, n_seq, d_hidn)
        return output